# Preliminaries

In [1]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

# PrefixSpan

### Project a sequence

In [ ]:
"""
Projects a sequence according to a given prefix, as done in PrefixSpan

Args:
    sequence: the sequence the projection is built from
    prefix: the prefix that is searched for in the sequence
    newEvent: if set to True, the first itemset is ignored
Returns:
    If the sequence does not contain the prefix, then None.
    Otherwise, a new sequence starting from the position of the prefix, including the itemset that includes the prefix
"""
def projectSequence(sequence, prefix, newEvent):
    result = None
    for i, itemset in enumerate(sequence):
        if result is None:
            if (not newEvent) or i > 0:
                if (all(x in itemset for x in prefix)):
                    result = [list(itemset)]
        else:
            result.append(copy.copy(itemset))
    return result

### Project a dataset

In [ ]:
"""
Projects a dataset according to a given prefix, as done in PrefixSpan

Args:
    dataset: the dataset the projection is built from
    prefix: the prefix that is searched for in the sequence
    newEvent: if set to True, the first itemset is ignored
Returns:
    A (potentially empty) list of sequences
"""
def projectDatabase(dataset, prefix, newEvent):
    projectedDB = []
    for sequence in dataset:
        seqProjected = projectSequence(sequence, prefix, newEvent)
        if not seqProjected is None:
            projectedDB.append(seqProjected)
    return projectedDB

### The main algorithm

#### Some more utility functions:

In [ ]:
"""
Generates a list of all items that are contained in a dataset
"""
def generateItems(dataset):
    return sorted(set ([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))

"""
Computes a defaultdict that maps each item in the dataset to its support
"""
def generateItemSupports(dataset, ignoreFirstEvent=False, prefix=[]):
    result = defaultdict(int)
    for sequence in dataset:
        if ignoreFirstEvent:
            sequence = sequence[1:]
        cooccurringItems = set()
        for itemset in sequence:
            if all(x in itemset for x in prefix):
                for item in itemset:
                    if not item in prefix:
                        cooccurringItems.add(item)
        for item in cooccurringItems:
            result [item] += 1
    return sorted(result.items())

#### Finally, the algorithm:

In [ ]:
"""
The PrefixSpan algorithm. Computes the frequent sequences in a seqeunce dataset for a given minSupport

Args:
    dataset: A list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: The minimum support that makes a sequence frequent
Returns:
    A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
"""
def prefixSpan(dataset, minSupport):
    result = []
    itemCounts = generateItemSupports(dataset)
    for item, count in itemCounts:
        if count >= minSupport:
            newPrefix = [[item]]
            result.append((newPrefix, count))
            result.extend(prefixSpanInternal(projectDatabase(dataset, [item], False), minSupport, newPrefix))
    return result

def prefixSpanInternal(dataset, minSupport, prevPrefixes=[]):
    result = []
    
    # Add a new item to the last element (==same time)
    itemCountSameEvent = generateItemSupports(dataset, False, prefix=prevPrefixes[-1])
    for item, count in itemCountSameEvent:
        if (count >= minSupport) and item > prevPrefixes[-1][-1]:
            newPrefix = copy.deepcopy(prevPrefixes)
            newPrefix[-1].append(item)
            result.append((newPrefix, count))
            result.extend(prefixSpanInternal(projectDatabase(dataset, newPrefix[-1], False), minSupport, newPrefix))
        
    # Add a new event to the prefix
    itemCountSubsequentEvents = generateItemSupports(dataset, True)
    for item, count in itemCountSubsequentEvents:
        if count >= minSupport:
            newPrefix = copy.deepcopy(prevPrefixes)
            newPrefix.append([item])
            result.append((newPrefix, count))
            result.extend(prefixSpanInternal(projectDatabase(dataset, [item], True), minSupport, newPrefix))
    return result

# Filter for closed and maximal patterns
### Closed patterns

In [ ]:
"""
Given a list of all frequent sequences and their counts, compute the set of closed frequent sequence (as a list)
This is only a very simplistic (naive) implementation for demonstration purposes!
"""
def filterClosed(result):
    for supersequence, countSeq in copy.deepcopy(result):
        for subsequence, countSubSeq in copy.deepcopy(result):
            if isSubsequence(supersequence, subsequence) and (countSeq == countSubSeq) and subsequence != supersequence:
                result.remove((subsequence, countSubSeq))

### Maximal sequences

In [ ]:
"""
Given a list of all frequent sequences and their counts, compute the set of maximal frequent sequence (as a list)
This is only a very naive implementation for demonstration purposes!
"""
def filterMaximal(result):
    for supersequence, countSeq in copy.deepcopy(result):
        for subsequence, countSubSeq in copy.deepcopy(result):
            if isSubsequence (supersequence, subsequence) and subsequence != supersequence:
                result.remove((subsequence, countSubSeq)) 

# Application 

Convert this to the list of list of lists that we use as a dataformat

In [2]:
import pandas as pd
df = pd.read_csv(r'/Users/clothildedevillenfagne/Cours/Master_2/Memoire/moyen_data.csv', sep="\t")

/Users/clothildedevillenfagne/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9,10,14,16,17,37,38,39,40,41,43,44,45,46,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
new_df = df[df['class']=='Aves']
new_df = new_df[['family','species', 'year', 'month', 'decimalLatitude', 'decimalLongitude', 'individualCount']]

In [ ]:
region = pd.read_csv('/Users/clothildedevillenfagne/Cours/Master_2/Memoire/region_obs_oiseaux.csv')
new_df['region']=region['0'].tolist()
new_df = new_df[new_df['region']=='Région de Bruxelles-Capitale']

In [ ]:
len(set(new_df['family'].tolist()))

### Regarder pour les espèce

In [ ]:
def ps_for_year(df):
    resultat_final=[]
    #df_use = df[df['year']==annee]
    year = set(df['year'].tolist())
    #gen = list(set(df['species'].tolist()))
    
    for ye in year:
        data1 = df[df.year == ye]
        print('année :', ye)
        #print(data1)
        month = set(data1['month'].tolist())
        new_month = []
        count = False
        for elem in month:
            if pd.isnull(elem):
                count = True
            else:
                new_month.append(elem)
            if count == True:
                new_month.append(float('nan'))
        month = new_month
        big_list =[]
        for mo in month:
            data2 = data1[data1.month == mo]
            longitude = set(data2['decimalLongitude'].tolist())
            print('mois :', mo)
            for lon in longitude:
                data3 = data2[data2.decimalLongitude == lon]
                latitude = set(data3['decimalLatitude'].tolist())
                for la in latitude:  # d'abord refaire un for pour longitude puis latitude
                    lst = []#[0] * len(gen) #np.nan
                    final_df = data3[data3.decimalLatitude == la]
                    #print(final_df)
                    # num_obs = len(final_df['decimalLatitude'])
                    genre = set(final_df['species'].tolist())
                    for elem in genre:
                        if pd.isnull(elem) == False:
                        #index = gen.index(elem)
                        #lst[index]+=1
                            lst.append(elem)

                    big_list.append(lst)
        wikispeediaData=[]
        for seq in big_list:
            newSeq = []
            for item in seq:
                newSeq.append([item])
            wikispeediaData.append(newSeq)
        min_sup = len(wikispeediaData)*10/100
        res = prefixSpan (wikispeediaData, min_sup)
        resultat = []
        for elem in res:
            resultat.append(elem+(ye,))
        resultat_final.extend(resultat)
    return resultat_final

In [ ]:
res = ps_for_year(new_df)

In [ ]:
res

## Traitement des résultat

In [ ]:
res[6][0][0][0]

In [ ]:
final =[] 
for i in range(len(res)):
    longueur = len(res[i][0])
    pattern = []
    for j in range(longueur):
        elem = res[i][0][j][0]
        pattern.append(elem)
        b = frozenset(pattern)
    c = (b, res[i][1], res[i][2])
    final.append(c)

In [ ]:
final

In [ ]:
df_prefix = pd.DataFrame(final, columns =['espece', 'compte', 'annee'])

In [ ]:
df_prefix

In [ ]:
ar = []
espece = set(df_prefix['espece'].tolist())
for element in espece:
    df_1 = df_prefix[df_prefix['espece']==element]
    list_annee = []
    list_count = []
    for el in df_1.itertuples():
        list_annee.append(el.annee)
        list_count.append(el.compte)
    list_pat = [element,df_1['compte'].sum() ,list_count, list_annee, len(set(list_annee))]
    ar.append(list_pat)
    

In [ ]:
df_prefixSpan = pd.DataFrame(ar, columns =['espèce', 'compte_total','compte', 'année', 'nombre_annee'])

In [ ]:
final_prefix = df_prefixSpan.sort_values(['nombre_annee', 'compte_total'], ascending = [False,False])

In [ ]:
final_prefix

In [ ]:
#final_prefix.to_csv('res_prefix.csv')

In [ ]:
df_prefixSpan.sort_values(by = 'nombre_annee', ascending = [False])['compte_total'].tolist()

In [ ]:
df_outte = df_prefixSpan[df_prefixSpan['espèce']== frozenset(['Alopochen aegyptiaca'])]

In [ ]:
count_outte = df_outte['compte'].tolist()
annee_ouette = df_outte['année'].tolist()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(annee_ouette[0],count_outte[0])
#plt.title("Nombre d'observation en fonction des années")
plt.xlabel('Années')
plt.ylabel("Nombre d'observation")
plt.savefig("obs_annee.png")
plt.show

## voir ratio nombre d'observation

In [ ]:
year = set(df['year'].tolist())
dico_year = {}
for ye in year:
    data1 = df[df['year']==ye]
    dico_year[ye]=len(data1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
plt.plot(dico_year.keys(),dico_year.values())
plt.title("Nombre d'observation en fonction des années")
plt.xlabel('Années')
plt.ylabel("Nombre d'observation)
plt.show

In [ ]:
new_dic = {}
count = 0
for key,value in dico_year.items():
    new_dic[key] = (count_outte[0][count]/value)*100
    count+=1

### pourcentage d'observation des ouette 

In [ ]:
new_dic

In [ ]:
plt.plot(new_dic.keys(),new_dic.values())
#plt.title("Ratio d'obseervation pour chaque année")
plt.xlabel('Années')
plt.ylabel("Observation(%)")
plt.savefig("rat_annee.png")
plt.show

### nombre d'ouette observer chaque année

In [ ]:
new_df

In [ ]:
year = set(df['year'].tolist())
dico_count = {}
for ye in year:
    data1 = df[df['year']==ye]
    dico_count[ye]=len(data1)
    

In [ ]:
dico_count

In [ ]:
year = set(new_df['year'].tolist())
new_new_df = new_df[new_df['species']=='Alopochen aegyptiaca']
moyen = new_new_df['individualCount'].sum()/len(new_new_df)
print('moyen : ', moyen)
dico_count_ou = {}
for ye in year:
    data1 = new_new_df[new_new_df['year']==ye]
    dico_count_ou[ye]=data1['individualCount'].sum()

In [ ]:
dico_count_ou

In [ ]:
plt.plot(dico_count_ou.keys(),dico_count_ou.values())
#plt.title("Ratio d'obseervation pour chaque année")
plt.xlabel('Années')
plt.ylabel("Nombre d'espèce")
#plt.savefig("rat_annee.png")
plt.show

## évaluer général, le Haematopus ostralegus (huitrier pie) 

In [ ]:
tri_df = df_prefixSpan[df_prefixSpan['nombre_annee']>=3]
espece = tri_df['espèce'].tolist()

In [ ]:
tri_df

In [ ]:
dico_croissance = {}
for elem in tri_df.itertuples():
    compte = elem.compte
    somme = 0
    count = 0
    for i in range(1, len(compte)):
        somme += (compte[i]-compte[i-1])/compte[i-1]
        count+=1
    moyen = somme/count
    dico_croissance[elem.espèce]=moyen

In [ ]:
dico_croissance

In [ ]:
df_croissance = pd.DataFrame(dico_croissance.items(), columns=['espèce', 'croissance'])
#print(df_croissance[df_croissance['croissance']>0.1])
df_croissance.sort_values(by = 'croissance', ascending = [False])

In [ ]:
year = set(df_outte['year'].tolist())
dico_year = {}
for ye in year:
    data1 = df_outte[df_outte['year']==ye]
    dico_year[ye]=len(data1)

In [ ]:
def evaluation (df ,df_prefixSpan, espece):
    df_outte = df_prefixSpan[df_prefixSpan['espèce']== frozenset([espece])]
    count_ouette = df_outte['compte'].tolist()
    annee_ouette = df_outte['année'].tolist()
    plt.figure(1)
    plt.plot(annee_ouette[0], count_ouette[0])
    plt.xlabel('Temps (Années)')
    plt.ylabel("Observation")
    plt.savefig("obs_annee.png")
    plt.show()
    print('ratio')
    year = set(df['year'].tolist())
    dico_year = {}
    for ye in year:
        data1 = df[df['year']==ye]
        dico_year[ye]=len(data1)
    new_dic = {}
    count = 0
    for key,value in dico_year.items():
        new_dic[key] = (count_ouette[0][count]/value)*100
        count+=1
    plt.figure(2)
    plt.plot(new_dic.keys(),new_dic.values())
    #plt.title("Ratio d'observation pour chaque année")
    plt.xlabel('Temps (Années)')
    plt.ylabel("Observation(%)")
    plt.savefig("rat_annee.png")
    plt.show()

In [ ]:
evaluation (new_df ,df_prefixSpan, 'Alopochen aegyptiaca')# Alopochen aegyptiaca 'Branta canadensis'

In [ ]:
df

# regarder taux de croissance famille bx

In [ ]:
new_df

In [ ]:
family = set(new_df['family'].tolist()) #63 famille différente 
taux_croissance = []
for fam in family:
    data1 = new_df[new_df['family']==fam]
    year = set(data1['year'].tolist())
    for ye in year:
        data2 = data1[data1['year']==ye]
        count = (data2['individualCount'].sum())/len(new_df[new_df['year']==ye]) # normalisation
        list_count = [fam, ye, count]
        taux_croissance.append(list_count)
df_taux = pd.DataFrame(taux_croissance, columns=['family', 'year', 'count'] )        

In [ ]:
df_taux[df_taux['family']=='Pelecanidae']#Passeridae Psittacidae

In [ ]:
df_taux_10 = df_taux[df_taux['year']>=2009]
df_taux_10 = df_taux_10[df_taux_10['year']<=2019]

In [ ]:
df_taux_10

In [ ]:
dico_croissance_neg = {}
dico_croissance_pos = {}
for fam in set(df_taux_10['family'].tolist()):
    data1 = df_taux_10[df_taux_10['family']==fam]
    count_list = data1['count'].tolist()
    somme = 0
    count = 1
    for i in range(1, len(count_list)):
        somme += (count_list[i]-count_list[i-1])/count_list[i-1]
        count+=1
    moyen = somme/count
    if moyen <0:
        dico_croissance_neg[fam]=(moyen, count)
    else:
        dico_croissance_pos[fam]=(moyen, count)

In [ ]:
dico_croissance_neg

In [ ]:
dico_croissance_pos

### Regarder pour les position

In [4]:
import folium
import numpy as np
from folium import plugins
from folium.plugins import HeatMap
import branca.colormap
from collections import defaultdict
import folium

In [55]:
new_df

,family,species,year,month,decimalLatitude,decimalLongitude,individualCount
477,Phasianidae,Phasianus colchicus,2021,4.0,51.22869,4.73774,1.0
479,Anatidae,Aix galericulata,2021,4.0,50.89379,4.76185,1.0
480,Anatidae,Alopochen aegyptiaca,2021,4.0,50.81495,4.29600,2.0
481,Anatidae,Cygnus atratus,2021,4.0,51.18572,3.27233,1.0
482,Anatidae,Alopochen aegyptiaca,2021,4.0,50.75811,5.17392,2.0
...,...,...,...,...,...,...,...
22563391,Rallidae,Fulica atra,2018,12.0,51.10804,4.17848,5.0
22563392,Accipitridae,Accipiter nisus,2018,12.0,51.24820,3.39401,1.0
22563393,Rallidae,Fulica atra,2018,12.0,50.88176,4.31492,8.0
22563394,Anatidae,Aythya fuligula,2018,12.0,50.97714,3.67660,32.0


In [ ]:
#df = new_df[['species', 'year', 'month', 'decimalLatitude', 'decimalLongitude']]

In [ ]:
#df

In [56]:
def ps_for_coo(df):
    latitude = df['decimalLatitude'].tolist()
    longitude = df['decimalLongitude'].tolist()
    coordonee = list(set(tuple(zip(latitude, longitude))))
    df_final = pd.DataFrame(index=coordonee)
    year = set(df['year'].tolist())
    for ye in year:
        print('Année : ', ye)
        #ye_list = []
        data1 = df[df['year']==ye]
        latitude = data1['decimalLatitude'].tolist()
        longitude = data1['decimalLongitude'].tolist()
        coordonee = list(set(tuple(zip(latitude, longitude))))
        big_list = []
        dim = len(data1)
        #for mo in month:
            #data2 = data1[data1.month == mo].copy()
        espece = set(data1['species'].tolist())
            #print('mois :', mo)
        for esp in espece:
            if pd.isnull(esp)==False:
                lst = [0] * len(coordonee)
                data2 = data1[data1['species'] == esp].copy()
                latitude_esp = data2['decimalLatitude'].tolist()
                longitude_esp = data2['decimalLongitude'].tolist()
                coordonee_esp = set(tuple(zip(latitude_esp, longitude_esp)))
                #list_esp = []
                for coo in coordonee_esp:
                    index = coordonee.index(coo)
                    lst[index] = 1
                big_list.append(lst)

        df_prefix_coo = pd.DataFrame(big_list, columns =coordonee)
        dico_coo = df_prefix_coo.sum()
        df_coo = pd.DataFrame(dico_coo, columns=[ye])
    
        df_final = pd.concat([df_final, df_coo], axis=1)
        
    
    return df_final 
        

In [57]:
res = ps_for_coo(new_df)

Année :  2000
Année :  2001
Année :  2002
Année :  2003
Année :  2004
Année :  2005
Année :  2006
Année :  2007
Année :  2008
Année :  2009
Année :  2010
Année :  2011
Année :  2012
Année :  2013
Année :  2014
Année :  2015
Année :  2016
Année :  2017
Année :  2018
Année :  2019
Année :  2020
Année :  2021


In [58]:
df_max_scaled = res.copy() 
  
for column in df_max_scaled.columns: 
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max() 
      
display(df_max_scaled) 

2000      2001      2002  2003      2004      2005  \
49.50731 5.46838       NaN       NaN       NaN   NaN       NaN       NaN   
49.50739 5.46773       NaN       NaN       NaN   NaN       NaN       NaN   
49.50783 5.46886       NaN       NaN       NaN   NaN       NaN       NaN   
49.50817 5.46647       NaN       NaN       NaN   NaN       NaN       NaN   
49.50958 5.47370       NaN       NaN       NaN   NaN       NaN       NaN   
...                    ...       ...       ...   ...       ...       ...   
51.50423 4.83707       NaN  0.121951       NaN   NaN       NaN       NaN   
         4.83708       NaN       NaN       NaN   NaN  0.022346  0.005587   
51.50512 4.77948       NaN       NaN  0.068966   NaN       NaN       NaN   
51.50533 4.76507  0.005814  0.292683       NaN   NaN       NaN       NaN   
51.50534 4.76507       NaN       NaN  0.057471   NaN  0.011173  0.011173   

                      2006  2007      2008      2009  ...      2012      2013  \
49.50731 5.46838       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50739 5.46773       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50783 5.46886       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50817 5.46647       NaN   NaN       NaN       NaN  ...  0.004785       NaN   
49.50958 5.47370       NaN   NaN       NaN       NaN  ...       NaN       NaN   
...                    ...   ...       ...       ...  ...       ...       ...   
51.50423 4.83707       NaN   NaN       NaN       NaN  ...       NaN       NaN   
         4.83708  0.011834   NaN  0.159624  0.199029  ...  0.177033  0.173709   
51.50512 4.77948       NaN   NaN       NaN       NaN  ...       NaN       NaN   
51.50533 4.76507       NaN   NaN       NaN       NaN  ...       NaN       NaN   
51.50534 4.76507       NaN   NaN  0.079812  0.140777  ...  0.119617  0.126761   

                      2014      2015      2016      2017      2018  2019  \
49.50731 5.46838       NaN       NaN  0.004695       NaN       NaN   NaN   
49.50739 5.46773       NaN       NaN       NaN       NaN       NaN   NaN   
49.50783 5.46886       NaN       NaN       NaN       NaN       NaN   NaN   
49.50817 5.46647       NaN       NaN       NaN       NaN       NaN   NaN   
49.50958 5.47370       NaN       NaN       NaN       NaN       NaN   NaN   
...                    ...       ...       ...       ...       ...   ...   
51.50423 4.83707       NaN       NaN       NaN       NaN       NaN   NaN   
         4.83708  0.349282  0.252525  0.230047  0.273585  0.232877   NaN   
51.50512 4.77948       NaN       NaN       NaN       NaN       NaN   NaN   
51.50533 4.76507       NaN       NaN       NaN       NaN       NaN   NaN   
51.50534 4.76507  0.177033  0.116162  0.178404  0.278302  0.269406   NaN   

                      2020      2021  
49.50731 5.46838       NaN       NaN  
49.50739 5.46773       NaN  0.111111  
49.50783 5.46886       NaN  0.111111  
49.50817 5.46647       NaN       NaN  
49.50958 5.47370  0.019608       NaN  
...                    ...       ...  
51.50423 4.83707       NaN       NaN  
         4.83708       NaN       NaN  
51.50512 4.77948       NaN       NaN  
51.50533 4.76507       NaN       NaN  
51.50534 4.76507       NaN       NaN  

[580498 rows x 22 columns]

In [59]:
def map_coo (df, annee):
    count = df[annee].tolist()
    coo = df.index.tolist()
    #coo = test_df['coordonee'].tolist()
    #compte = test_df['compte'].tolist()
    
    loc = [4.35, 50.8333]
    mappy = folium.Map(location=[loc[1], loc[0]], zoom_start=8)
    heat = []
    for i in range(len(coo)):
        if pd.isnull(count[i])==False:
            value = int(count[i]*2) #mettre 5 pour la deuxième analyse
            for j in range(value):
                heat.append(list(coo[i]))
    steps=20
    colormap = branca.colormap.linear.YlOrRd_09.scale(0, 1).to_step(steps)
    gradient_map=defaultdict(dict)
    for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    colormap.add_to(mappy)
    HeatMap(heat,gradient = gradient_map).add_to(mappy)

    return mappy

In [ ]:
map_coo(df_max_scaled,2000)

In [ ]:
map_coo(df_max_scaled,2001)

In [ ]:
map_coo(df_max_scaled,2002)

In [ ]:
map_coo(df_max_scaled,2003)

In [ ]:
map_coo(df_max_scaled,2004)

In [ ]:
map_coo(df_max_scaled,2005)

In [60]:
map_coo(df_max_scaled,2006)

In [ ]:
map_coo(df_max_scaled,2007)

In [ ]:
map_coo(df_max_scaled,2008)

In [ ]:
map_coo(df_max_scaled,2009)

In [ ]:
map_coo(df_max_scaled,2010)

In [ ]:
map_coo(df_max_scaled,2011)

In [ ]:
map_coo(df_max_scaled,2012)

In [ ]:
map_coo(df_max_scaled,2013)

In [ ]:
map_coo(df_max_scaled,2014)

In [ ]:
map_coo(df_max_scaled,2015)

In [ ]:
map_coo(df_max_scaled,2016)

In [ ]:
map_coo(df_max_scaled,2017)

In [ ]:
map_coo(df_max_scaled,2018)

In [61]:
map_coo(df_max_scaled,2019)

# Analyse nombre

In [8]:
def ps_for_num(df):
    latitude = df['decimalLatitude'].tolist()
    longitude = df['decimalLongitude'].tolist()
    coordonee = list(set(tuple(zip(latitude, longitude))))
    df_final = pd.DataFrame(index=coordonee)
    year = set(df['year'].tolist())
    for ye in year:
        print('Année : ', ye)
        #ye_list = []
        data1 = df[df['year']==ye]
        latitude = data1['decimalLatitude'].tolist()
        longitude = data1['decimalLongitude'].tolist()
        coordonee = list(set(tuple(zip(latitude, longitude))))
        big_list = []
        dim = len(data1)
        #for mo in month:
            #data2 = data1[data1.month == mo].copy()
        espece = set(data1['species'].tolist())
            #print('mois :', mo)
        for esp in espece:
            if pd.isnull(esp)==False:
                lst = [0] * len(coordonee)
                data2 = data1[data1['species'] == esp].copy()
                latitude_esp = data2['decimalLatitude'].tolist()
                longitude_esp = data2['decimalLongitude'].tolist()
                coordonee_esp = tuple(zip(latitude_esp, longitude_esp))
                ind_count = data2['individualCount'].tolist()
                for i in range(len(coordonee_esp)):
                    index = coordonee.index(coordonee_esp[i])
                    lst[index] += ind_count[i]
                big_list.append(lst)

        df_prefix_coo = pd.DataFrame(big_list, columns =coordonee)
        dico_coo = df_prefix_coo.sum()
        df_coo = pd.DataFrame(dico_coo, columns=[ye])
        #dico_final={}
        #for key, value in dico_coo.items():
        #    dico_final[key]=value/dim
        #print(dico_final)
        df_final = pd.concat([df_final, df_coo], axis=1)
        #print(df_final)
    
    return df_final #resultat_final
        

In [9]:
res2 = ps_for_num(new_df)

Année :  2000
Année :  2001
Année :  2002
Année :  2003
Année :  2004
Année :  2005
Année :  2006
Année :  2007
Année :  2008
Année :  2009
Année :  2010
Année :  2011
Année :  2012
Année :  2013
Année :  2014
Année :  2015
Année :  2016
Année :  2017
Année :  2018
Année :  2019
Année :  2020
Année :  2021


In [10]:
df_max_scaled = res2.copy() 
  
for column in df_max_scaled.columns: 
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max() 
      
display(df_max_scaled) 

2000      2001     2002  2003      2004      2005  \
49.50731 5.46838       NaN       NaN      NaN   NaN       NaN       NaN   
49.50739 5.46773       NaN       NaN      NaN   NaN       NaN       NaN   
49.50783 5.46886       NaN       NaN      NaN   NaN       NaN       NaN   
49.50817 5.46647       NaN       NaN      NaN   NaN       NaN       NaN   
49.50958 5.47370       NaN       NaN      NaN   NaN       NaN       NaN   
...                    ...       ...      ...   ...       ...       ...   
51.50423 4.83707       NaN  0.000881      NaN   NaN       NaN       NaN   
         4.83708       NaN       NaN      NaN   NaN  0.000058  0.000003   
51.50512 4.77948       NaN       NaN  0.00000   NaN       NaN       NaN   
51.50533 4.76507  0.000006  0.000940      NaN   NaN       NaN       NaN   
51.50534 4.76507       NaN       NaN  0.00009   NaN  0.000116  0.000060   

                      2006  2007      2008      2009  ...      2012      2013  \
49.50731 5.46838       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50739 5.46773       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50783 5.46886       NaN   NaN       NaN       NaN  ...       NaN       NaN   
49.50817 5.46647       NaN   NaN       NaN       NaN  ...  0.000004       NaN   
49.50958 5.47370       NaN   NaN       NaN       NaN  ...       NaN       NaN   
...                    ...   ...       ...       ...  ...       ...       ...   
51.50423 4.83707       NaN   NaN       NaN       NaN  ...       NaN       NaN   
         4.83708  0.000015   NaN  0.001148  0.001927  ...  0.001073  0.000592   
51.50512 4.77948       NaN   NaN       NaN       NaN  ...       NaN       NaN   
51.50533 4.76507       NaN   NaN       NaN       NaN  ...       NaN       NaN   
51.50534 4.76507       NaN   NaN  0.000271  0.000882  ...  0.000141  0.000376   

                      2014      2015      2016      2017      2018  2019  \
49.50731 5.46838       NaN       NaN  0.000003       NaN       NaN   NaN   
49.50739 5.46773       NaN       NaN       NaN       NaN       NaN   NaN   
49.50783 5.46886       NaN       NaN       NaN       NaN       NaN   NaN   
49.50817 5.46647       NaN       NaN       NaN       NaN       NaN   NaN   
49.50958 5.47370       NaN       NaN       NaN       NaN       NaN   NaN   
...                    ...       ...       ...       ...       ...   ...   
51.50423 4.83707       NaN       NaN       NaN       NaN       NaN   NaN   
         4.83708  0.001431  0.000816  0.000521  0.001197  0.001649   NaN   
51.50512 4.77948       NaN       NaN       NaN       NaN       NaN   NaN   
51.50533 4.76507       NaN       NaN       NaN       NaN       NaN   NaN   
51.50534 4.76507  0.000254  0.000172  0.000206  0.000543  0.000933   NaN   

                      2020      2021  
49.50731 5.46838       NaN       NaN  
49.50739 5.46773       NaN  0.000481  
49.50783 5.46886       NaN  0.000962  
49.50817 5.46647       NaN       NaN  
49.50958 5.47370  0.000004       NaN  
...                    ...       ...  
51.50423 4.83707       NaN       NaN  
         4.83708       NaN       NaN  
51.50512 4.77948       NaN       NaN  
51.50533 4.76507       NaN       NaN  
51.50534 4.76507       NaN       NaN  

[580498 rows x 22 columns]

In [34]:
map_coo(df_max_scaled,2000)

In [35]:
map_coo(df_max_scaled,2001)

In [36]:
map_coo(df_max_scaled,2002)

In [37]:
map_coo(df_max_scaled,2003)

In [38]:
map_coo(df_max_scaled,2004)

In [39]:
map_coo(df_max_scaled,2005)

In [40]:
map_coo(df_max_scaled,2006)

In [41]:
map_coo(df_max_scaled,2007)

In [42]:
map_coo(df_max_scaled,2008)

In [43]:
map_coo(df_max_scaled,2009)

In [44]:
map_coo(df_max_scaled,2010)

In [45]:
map_coo(df_max_scaled,2011)

In [46]:
map_coo(df_max_scaled,2012)

In [47]:
map_coo(df_max_scaled,2013)

In [48]:
map_coo(df_max_scaled,2014)

In [49]:
map_coo(df_max_scaled,2015)

In [50]:
map_coo(df_max_scaled,2016)

In [51]:
map_coo(df_max_scaled,2017)

In [52]:
map_coo(df_max_scaled,2018)

In [54]:
map_coo(df_max_scaled,2019)